
# Natural Language Processing


## 2. Naive Bayes i Atribució d'autors


En aquesta segona part es programarà un classificador, que donat un document el categoritzarà per atribuir-lo al seu possible autor. Els textos provindran del dataset amb el qual s'ha treballat a la primera part.


**Què s’ha de fer?**

Volem classificar textos literaris segons el seu escriptor. 
A partir de tots els textos que tenim, desenvoluparem un classificador probabilístic del tipus Naive Bayes que ens permeti identificar a quin autor pertany llibre o text segons les característiques triades.


**Quina és la idea del sistema de classificació que s’ha de desenvolupar?**

El classificador és un concepte de l'aprenentatge automàtic supervisat. 
L'objectiu del classificador és donat unes característiques que descriuen els objectes que es volen classificar indicar a quina categoria o classe pertanyen d'entre un conjunt predeterminat. 
El procés de classificació consta de dues parts: 
(a) el procés d'aprenentatge i 
(b) el procés d'explotació o testeig. 
El procés d'aprenentatge rep exemples de parelles $(x,y)$ on $x$ són les característiques, usualment nombres reals, i $y$ és la categoria a la que pertanyen. 
Aquest conjunt se'l coneix com a conjunt d'entrenament i ens servirà per trobar una funció $\hat{y}=h(x)$ que donada una $x$ aconsegueixi que $\hat{y}=y$. Per altra banda el procés de testeig aplica la funció $h(x)$ apresa a l'entrenament a una nova descripció per veure quina categoria li correspon.


**Classificació i llenguatge natural**

La descripció dels exemples en característiques és el punt més crític de tot sistema d'aprenentatge automàtic. 
Una de les representacions més simples per tal de descriure un text és la representació *bag-of-words*.
Aquesta representació converteix un text en un conjunt de $N$ paraules. 
Consisteix en seleccionar un conjunt d'$N$ paraules i per cada paraula comptar quants cops apareix en el text. 
Una versió alternativa d'aquest procés pot ser simplement indicar si apareix o no en el text. 


## Abans de començar


**\+ No es poden modificar les definicions de les funcions donades, ni canviar els noms de les variables i paràmetres ja donats**


**\+ En les funcions, s'especifica què serà i de quin tipus cada un dels paràmetres, cal respectar-ho**
 

Executeu les següents dues cel·les: inclouen alguns imports necessaris, així com la funció `tokenize` per separar paraules.



In [ ]:
import re
from collections import Counter, defaultdict
from glob import glob
import os
from math import log

In [ ]:
def tokenize(text, lowercase=True):
    text = text.lower() if lowercase else text
    for match in re.finditer(r"\w+(\.?\w+)*", text):
        yield match.group()

list(tokenize('taller dels nous usos de la informàtica'))

### N-grams

Per fer l'exercici més interessant, generalitzarem una mica la representació de bag-of-words per acceptar n-grames. Si tenim la frase "taller de nous usos de la informàtica", els 1-grames són cadascuna de les paraules individuals. En canvi, els bigrames (2-grames) són les parelles de paraules consecutives:

```
"taller de", "de nous", "nous usos", "usos de", "de la", "la informàtica".
```

Els n-grames guarden més informació d'un text i de l'estil de qui l'ha escrit.

**Exercici 1**

Implementeu la funció que donat un iterable ens dóna la seva llista de n-grames.

In [ ]:
def ngrams(iterable, ngram_range=(1, 1)):
    """Donat un iterable (típicament, serà una llista de paraules),
    retornar els seus ngrames, des dels de menor longitud als de major longitud,
    segons ngram_range.
    
    Input:
        iterable: una llista de paraules
        ngram_range: tupla amb dos elements: la longitud mínima i la màxima dels 
            ngrames que volem
    
    Returns:
        Llista amb tots els ngrames generats
    
    """
    min_n, max_n = ngram_range
    # Passem els iterables a tuples perque siguin 'hashable'
    if isinstance(iterable, (list, set)):
        iterable = tuple(iterable)
    ngrams = []
    
    # part a implementar per l'alumne
    # EL VOSTRE CODI AQUÍ
    return ngrams

In [ ]:
# Els següents tests haurien de donar `True` si ho heu implementat correctament:
ngrams(
    list(tokenize("taller de nous usos de la informàtica")), 
    ngram_range=(1, 2)
) == [
    ('taller',), ('de',), ('nous',), ('usos',), ('de',), ('la',), ('informàtica',), 
    ('taller', 'de'), ('de', 'nous'), ('nous', 'usos'), 
    ('usos', 'de'), ('de', 'la'), ('la', 'informàtica')
]

### Processament de les dades: generació de features amb tokens i amb n-grames

El primer que farem és processar cadascun dels textos que volem considerar. Per cada document, el convertirem a una llista de paraules i en generarem n-grames. Després, seleccionarem els N n-grames més comuns. Aquests seran les "features" del document. En altres paraules, treballem amb la hipòtesi que els n-grames més comuns determinen l'estil de l'autor. 

*Recordeu que podeu utilitzar la classe `Counter`.*

**Exercici 2.** Implementeu la funció `create_features` tal com s'ha descrit.

In [ ]:
def create_features(filenames, N=10, ngram_range=(1,1)):
    """A partir d'una llista d'arxius i un rang d'ngrames, creem dos 
    llistes: un de noms d'autor, i un de llistes de ngrames.
        
    Input:
        filenames: llista de documents a processar
        ngram_range: n-grames que volem crear
        
    Returns:
        X: llista dels N ngrames més comuns de cada document
        y: llista d'autors (etiquetes) de cada document
    """
    
    X = []
    y = []
    
    for filename in filenames:
        author = (os.path.basename(filename).split('-'))[0]
        words = list(tokenize(open(filename, encoding="utf-8").read()))
    
        # part a implementar per l'alumne
        # EL VOSTRE CODI AQUÍ
    
    return X, y

In [ ]:
train_X, train_y = create_features(glob('data/gutenberg/training/*.txt'), N=10)
print(train_y[0], train_X[0])

## Naive bayes

### El classificador Naïve Bayes

Un cop tenim una representació necessitem un procés d'aprenentatge que ens permeti passar de la descripció a una categoria. 
En aquest lliurament farem servir el classificador Naïve Bayes. 
Aquest classificador forma part de la família de classificadors probabilístics. 
La sortida d'un classificador probabilístic és un valor de probabilitat donat un exemple per cadascuna de les categories. 
La decisió final correspon a la categoria amb més probabilitat. 


Els classificadors probabilistics Bayesians es basen en el teorema de Bayes per realitzar els càlculs per trobar la probabilitat condicionada: 
$$ p(x,y) = p(x|y)p(y) = p(y|x)p(x)$$
d'on podem extreure que: 
$$ p(y|x) = \frac{p(x|y)p(y)}{p(x)}$$


En molts casos $p(y)$ i $p(x)$ són desconeguts i es consideren equiprobables. Per nosaltres, p(y) serà la quantitat de vegades que hem vist l'autor y dividit pel nombre total de documents. Aquesta distribució de probabilitat dels autors de vegades s'anomena la "prior".
Per tant, la decisió es simplifica a:
$$ p(y|x) = c · p(x|y)p(y).$$


Les deduccions fins a aquest punt són vàlides per la majoria de classificadors Bayesians. 
Naïve Bayes es distingeix de la resta perquè imposa una condició encara més restrictiva. 
Considerem $x=(x_1, \cdots, x_n)$ un conjunt d'$n$ variables aleatòries. 
Naïve Bayes assumeix que totes elles són independents entre elles i per tant podem escriure:
$$p(x_1,x_2,...,x_n | y) = p(x_1|y)p(x_2|y)...p(x_n|y).$$
Podem interpretar l'anterior equació de la següent forma: La probabilitat de que el tweet descrit pel vector de característiques ("in the", "of the", "and") sigui de la classe "blake" és proporcional al producte de la probabilitat que la "in the" del vector aparegui en els textos de "blake" per la probabilitat que "of the" hi aparegui, per la probabilitat que "and" hi aparegui.


La fórmula final queda doncs
$$ p(y|x_1,x_2,\dots,x_N) \propto p(x_1|y)p(x_2|y)\cdots p(x_n|y)p(y)$$
on $y$ és un autor i $x_1,\dots,x_N$ és un conjunt de paraules o n-grames.


**Estimant les probabilitats marginals condicionades**

L'últim pas que ens queda és trobar el valor de les probabilitats condicionades. 
Per trobar el valor de la probabilitat condicionada farem servir una aproximació freqüentista a la probabilitat. 
Això vol dir que calcularem la freqüència d'aparició de cada paraula per a cada autor. 
Aquest càlcul es fa dividint el nombre de textos de l'autor en què apareix la paraula pel nombre total de textos d'aquell autor. 

En general:
$$p(x = \text{"father"} | y = C)= \frac{A}{B} $$
on A és el número de textos de l'autor C on hi apareix la paraula 'father' i B és el número total de textos de l'autor C.


### Punts dèbils:

**El problema de la probabilitat 0**

Si us hi fixeu bé, la probabilitat pot ser 0 !! 
Això vol dir, que si en el text hi apareix una paraula que no hem vist abans, no pot ser classificada per cap autor.
El que s'acostuma a fer és donar una baixa probabilitat en comptes de zero. 
Una de les possibles solucions es fer servir la correcció de Laplace. 
Seguint l'exemple anterior la correcció de Laplace és
$$p(x= \text{"father"} | y = 'C' ) = \frac{A+1}{B+M}$$ 
on M és el nombre de categories, i A, B, C són com abans.

**El problema del "underflow"**

La funció que hem de calcular en el Naive Bayes és un producte. 
El nombre de característiques del vector és el nombre de termes del producte. 
Aquests nombres són iguals o menors a 1, si els multipliquem tots entre ells el resultat serà massa petit per a representar-lo en un float i el càlcul acabarà sent reduït a zero. 
Per solucionar aquest problema en comptes d'operar fent multiplicacions, se sol passar a l'escala logarítmica i allà operar fent servir sumes en comptes de multiplicacions. Dit d'una altra manera,

$$\log(p(y|x_1,x_2,\dots,x_N)) \propto \log(p(y)) + \log(p(x_1|y)) + \log(p(x_2|y))+\cdots+ \log(p(x_n|y))$$

### Classificar:

Donat un llistat de n-grames $x=(x_1,...,x_n)$, per classificar el corresponent text calcularem la probabilitat de pertànyer a cadascun dels autors:

$$p(\text{austen}|x) = p(\text{austen})\prod_{i=1}^np(x_i|\text{austen})$$
$$\cdots$$
$$p(\text{shakespeare}|x) = p(\text{shakespeare})\prod_{i=1}^np(x_i|\text{shakespeare})$$

I finalment, el text és de l'autor de probabilitat màxima. 

**Exercici 3.** Implementeu la classe `NaiveBayesLearner`. Us donem el mètode `fit`, que incorpora vectors de features i etiquetes per calcular les probabilitats. Heu d'implementar tres mètodes: 

- prior
- probability
- predict

Les cel·les posteriors us permeten testejar la vostra implementació.

In [ ]:
class NaiveBayes():
    """Classificador Naive Bayes
    """

    def fit(self, X, y):
        """Entrenament del Naive Bayes Classifier.
        
        Input:
            X: llista de features (cada feature serà un vector de n-grames)
            y: llista d'etiquetes (autors). L'ordre de X i de y ha de correspondre
        """
        self.C = Counter(y)
        self.N = defaultdict(Counter)
        for x, y_x in zip(X, y):
            self.N[y_x] += Counter(x)
        self.V = len(set(x for y_x in self.N for x in self.N[y_x]))

        
    def prior(self, y):
        """Donada una etiqueta y, retorna p(y), 
        segons la informació que tenim a self.C
        
        Input:
            y: string, una etiqueta (autor)
        
        Returns:
            p(y), float entre 0 i 1
        """
        
        raise NotImplementedError()

        
    def probability(self, x, y):
        """Dona la probabilitat que ens aparegui l'n-grama x si l'etiqueta és y: p(x|y)
        
        Input:
            x: un n-grama
            y: una etiqueta (autor)
        
        Returns:
            p(x|y), float entre 0 i 1
        """

        raise NotImplementedError()
        
        
    def predict(self, x):
        """Prediu l'etiqueta de l'exemple x (un vector de n-grames).
        
        Input:
            x: llista de n-grames
            
        Returns:
            L'etiqueta predita per x, d'acord amb la fórmula del Naive Bayes
        """
        
        raise NotImplementedError()

In [ ]:
N = 10
train_X, train_y = create_features(glob('data/gutenberg/training/*.txt'), N=N, ngram_range=(1,1))
test_X, test_y = create_features(glob('data/gutenberg/testing/*.txt'), N=N, ngram_range=(1,1))

bayes = NaiveBayes()
bayes.fit(train_X, train_y)

In [ ]:
predictions = []
for x, y in zip(test_X, test_y):
    x_counter = Counter(x)

    y_pred = bayes.predict(x)
    predictions.append(y_pred)
    print(y_pred, y)

### Avaluació de resultats: accuracy, precision, recall

En aquesta darrera secció avaluarem el nostre classificador i provarem de trobar un bon conjunt de paràmetres (mida dels ngrames, nombre N de ngrames més comuns als vectors de features) per a fer-lo millor.

Suposem que tenim un seguit d'atribucions correctes $t_1,\dots,t_n$ i una llista de prediccions d'autors $y_1,\dots,y_n$. El més senzill que podem fer per saber si ho hem fet és mesurar l'*accuracy*:

$$\operatorname{accuracy}\left(y_{1}, y_{2}, \ldots, y_{n}\right)=\frac{1}{n} \sum_{i=1}^{n} \begin{cases}1 & \text { si } y_{i} \text { és igual a } t_{i} \\ 0 & \text { en cas que no }\end{cases}.$$

**Exercici 4.** Implementeu la funció d'accuracy.

In [ ]:
def accuracy(pred_labels, true_labels):
    """Càlcul de la funció d'accuracy
    
    Input:
        pred_labels: llista d'etiquetes predites
        true_labels: llista d'etiquetes correctes
        
    Returns:
        accuracy(pred_labels)
    """
    
    raise NotImplementedError()

In [ ]:
print(accuracy(predictions, test_y))

Aquesta mesura de l'accuracy no ens diu massa cosa, ja que estem fent molt poques comprovacions. 

Una bona forma de veure com funcionaria el nostre classificador davant de dades sobre les quals no s'ha entrenat és fer servir l'estratègia n-fold. Aquesta estratègia testeja el classificador amb una partició de les dades d'entrenament i fa l'entrenament sobre la resta de dades que hem exclòs. Aquest procés d'exclusió es repeteix per cadascún dels *folds de les dades d'entrenament. El nombre de folds determina quantes particions hem de fer, i per tant, les dades que hi ha en el conjut de test. 

Per exemple, en un 5-fold validation, es fan 5 particions, les dades de test són un cinquè de les dades i l'entrenament es fa amb els quatre cinquens restants. El percentatge d'errors fent servir aquesta estratègia permet comparar classificadors.


In [ ]:
def train_and_test(X, y, start, end):
    """Funció per entrenar i testejar un classificador. Reservem
    X[start:end] per entrenar, fem el test sobre la resta d'exemples.
    
    Inputs:
        X: llista de vectors de característiques
        y: llista d'etiquetes de cada exemple
        start, end: posicions per reservar a l'entrenament
    
    Return:
        proporció d'exemples correctament predits pel classificador
    """
    
    train_X, train_y = X[:start] + X[end:], y[:start] + y[end:]
    test_X, test_y = X[start:end], y[start:end]
    
    bayes = NaiveBayes()
    bayes.fit(train_X, train_y)
    predictions = []
    for x, y in zip(test_X, test_y):
        predictions.append(bayes.predict(x))
        
    return accuracy(predictions, test_y)

In [ ]:
import random

def shuffle(*args, seed=None):
    """Desordenació aleatòria d'exemples d'entrenament (X,y),
    respectant la concordància entre cada exemple i la seva etiqueta
    """
    data = list(zip(*args))
    random.shuffle(data, random=seed)
    return zip(*data)

def n_fold(X, y, k=10):
    """Validació n-fold
    """
    if k is None: k = len(X)
    n = len(X)
    X, y = shuffle(X, y)
    scores = []
    for i in range(k):
        start, end = int(i * (n / k)), int((i + 1) * (n / k))
        score = train_and_test(X, y, start, end)
        scores.append(score)
    return sum(scores) / len(scores)

N=10
X, y = create_features(glob('data/gutenberg/**/*.txt'), N=N, ngram_range=(1,1))

In [ ]:
print(n_fold(X, y))

**Exercici 5.** Trobeu una combinació de `N` i `ngram_range` per tal que l'accuracy mitjana del `n_fold(X, y)` sigui com a mínim del 80%.

In [ ]:
raise NotImplementedError()